## Analyzing dome monitor with a labJack T7
Uses 2 analog input (AINs) to read the data.

Craig Lage - Jul 5, 2023

In [ ]:
import sys, os, re
import time, datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
dir = "/scratch/labJackData/"

files = os.listdir(dir)
counter = 0
for file in files:
    split_file = re.split('_|T|\Z.', file)
    if split_file[0] == "Dome" and split_file[1] == "Main":
        if (split_file[2] == '20230905'):# or \
            #(split_file[1] == '20230830' and int(list(split_file[2])[0]) > 0):
            filename = dir+file

            pkl_file = open(filename, 'rb')
            df = pkl.load(pkl_file)
            pkl_file.close()
            print(filename, len(df), df.columns)
            if counter == 0:
                merged_df = df
            else:
                merged_df = pd.concat([merged_df, df])
            #if counter == 2:
            #    break
            counter += 1


In [ ]:
print(len(df), len(merged_df))

In [ ]:
merged_df.columns

In [ ]:
client = EfdClient('summit_efd')
start = Time("2023-08-30T14:00:00Z", scale='utc')
end = Time("2023-08-31T01:00:00Z", scale='utc')
state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)


In [ ]:
fig, ax = plt.subplots(1,1)
merged_df["AIN0"].plot(ax=ax, label="Close button", color='red')

In [ ]:
print(len(state), len(df), len(merged_df))

In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])


merged_df["AIN0"].plot(ax=ax, label="Close button", color='red')
merged_df["AIN1"].plot(ax=ax, label="Motor relay", color='green')
merged_df["AIN2"].plot(ax=ax, label="Rain sensor", color='magenta')
ax.set_ylim(-0.5,10.0)
ax.set_title("AuxTel Dome Closing Monitor 20230829", fontsize=16)

ax.legend(loc='upper left')
#for start_time in start_times:
#    ax.axvline(start_time, ls='--', color='black')
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_30Aug23.png")

In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State', ls='--')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])
start_plot = Time("2023-08-29T21:30:00Z", scale='utc')
end_plot = Time("2023-08-29T23:55:00Z", scale='utc')

merged_df["AIN0"].plot(ax=ax, label="Close button", color='red')
merged_df["AIN1"].plot(ax=ax, label="Motor relay", color='green')
merged_df["AIN2"].plot(ax=ax, label="Rain sensor", color='magenta')
ax.set_title("AuxTel Dome Closing Monitor 20230824", fontsize=16)
ax.set_ylim(-0.5,10.0)
ax.set_xlim(start_plot.isot, end_plot.isot)
ax.legend()
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_BlowUp_29Aug23.png")

In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State', marker='x')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])
start_plot = Time("2023-08-29T23:43:36Z", scale='utc')
end_plot = Time("2023-08-29T23:43:37Z", scale='utc')

merged_df["AIN0"].plot(ax=ax, label="Close button", color='red')
merged_df["AIN1"].plot(ax=ax, label="Motor relay", color='green')
merged_df["AIN2"].plot(ax=ax, label="Rain sensor", color='magenta')
ax.set_title("AuxTel Dome Closing Monitor 20230824", fontsize=16)
ax.set_ylim(-2.0,10.0)
ax.set_xlim(start_plot.isot, end_plot.isot)
ax.legend()
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_BlowUp_6_29Aug23.png")

In [ ]:
# Find the times of the events
close_df = merged_df[merged_df['AIN1'] > 6.0]
start_times = []
end_times = []
durations = []
for i in range(len(close_df)):
    if i == 0:
        start_time = close_df.index[0]
        continue
    if Time(close_df.index[i].isoformat()).unix_tai - Time(close_df.index[i-1].isoformat()).unix_tai < 0.05:
        continue
    else:
        end_time = close_df.index[i-1]
        duration = Time(end_time.isoformat()).unix_tai - Time(start_time.isoformat()).unix_tai
        print(start_time, end_time, duration)
        start_times.append(start_time)
        end_times.append(end_time)
        durations.append(duration)        
        start_time = close_df.index[i]


In [ ]:
print(len(durations))
plt.title("Phantom close durations 20230829", fontsize=16)
plt.hist(durations, range=(0,0.25))
plt.xlabel("Duration(seconds)")
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_Durations_29Aug23.png")

## The code below fills in the state DataFrame with the last state

In [ ]:
cols = list(state.columns)
cols.remove('state')
newState = state.drop(cols, axis=1)

In [ ]:
times = []
vals = []
values = newState['state'].values
for i in range(len(newState)-1):
    n = int((state.index[i+1] - state.index[i]).total_seconds() * 100)
    #print(n)
    #continue
    value = values[i]
    for j in range(n):
        times.append(state.index[i] + pd.Timedelta(milliseconds=(j+1)*10))
        vals.append(value)
    

In [ ]:
holdState = pd.DataFrame(vals, columns=['state'], index=times)

In [ ]:
len(holdState)

In [ ]:
holdState.head(3)

In [ ]:
plotState = pd.concat([newState, holdState])

In [ ]:
# Plot blow-ups of the events
pdf = PdfPages("/home/craiglagegit/DATA/Dome_Monitor_Plots_Test_29Aug23.pdf")
fig = plt.figure(figsize=(8,8))

 
for i in range(len(start_times)):
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax2 = ax.twinx()
    ax2.set_ylim(0,6)
    ax2.set_yticks([1,2,3,4,5])
    ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])
    start_plot = Time(start_times[i].isoformat(), scale='utc') - TimeDelta(0.5, format='sec')
    end_plot = Time(end_times[i].isoformat(), scale='utc') + TimeDelta(0.5, format='sec')
    sub_df = merged_df.loc[(merged_df.index >= start_plot.isot) & (merged_df.index <= end_plot.isot)]
    sub_state = plotState.loc[(plotState.index >= start_plot.isot) & (plotState.index <= end_plot.isot)]
    sub_state['state'].plot(ax=ax2, color='blue', label='Actual State')
    print(start_plot, end_plot)
    sub_df["AIN0"].plot(ax=ax, label="Close button", color='red')
    sub_df["AIN1"].plot(ax=ax, label="Motor relay", color='green')
    sub_df["AIN2"].plot(ax=ax, label="Rain sensor", color='magenta')
    timestamp = start_times[i].isoformat().split('.')[0].replace('-','').replace(':','')
    ax.set_title(f"AuxTel Dome Closing Monitor {timestamp}", fontsize=16)
    ax.set_ylim(-2.0,10.0)
    ax.set_xlim(start_plot.isot, end_plot.isot)
    ax.text(start_plot.isot, 5.0, f"  Duration={durations[i]:.4f} sec.")
    ax.legend(loc='upper right')
    ax2.legend(loc='upper left')
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()
